In [15]:
import os
import pandas as pd
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain_groq import ChatGroq
import groq
from langchain_experimental.agents import create_csv_agent
from langchain.agents.agent_types import AgentType
from dotenv import load_dotenv

In [41]:
load_dotenv()

True

In [42]:
groq_client = groq.Client()

In [18]:
from typing import Any

def custom_groq_agent(df: pd.DataFrame, **kwargs) -> Any:
    # Process the DataFrame using Groq API
    response = groq_client.process_dataframe(df)
    return response

# Mock function for demonstration (replace with actual API call)
class MockGroqClient:
    def process_dataframe(self, df):
        # Example processing (replace with actual logic)
        return df.describe()

groq_client = MockGroqClient()

In [24]:
# Use the custom agent function with Hugging Face tools
pd_agent = create_pandas_dataframe_agent(ChatGroq(temperature=0.1), df, verbose=True, allow_dangerous_code=True)


In [25]:
df = pd.read_csv('train_expanded.csv')
df[0:10]

,question,answer
0,How can I create an account?,"To create an account, click on the 'Sign Up' b..."
1,What payment methods do you accept?,"We accept major credit cards, debit cards, and..."
2,How can I track my order?,You can track your order by logging into your ...
3,What is your return policy?,Our return policy allows you to return product...
4,Can I cancel my order?,You can cancel your order if it has not been s...
5,How long does shipping take?,Shipping times vary depending on the destinati...
6,Do you offer international shipping?,"Yes, we offer international shipping to select..."
7,What should I do if my package is lost or dama...,If your package is lost or damaged during tran...
8,Can I change my shipping address after placing...,"If you need to change your shipping address, p..."
9,How can I contact customer support?,You can contact our customer support team by p...


In [38]:
pd_agent.run("Can I order a product for delivery to a different country?")



> Entering new AgentExecutor chain...
Thought: I need to check if there is any information about international shipping in the dataframe.
Action: python_repl_ast
Action Input: df[df['question'].str.contains('country')]                                             question  \
33  Can I order a product for delivery to a differ...   

                                               answer  
33  Yes, we offer international shipping to select...  I now know the final answer
Final Answer: Yes, the dataframe indicates that you can order a product for delivery to a different country. Specific instructions for international shipping can be found by searching the answer column for "country".

> Finished chain.


'Yes, the dataframe indicates that you can order a product for delivery to a different country. Specific instructions for international shipping can be found by searching the answer column for "country".'

In [40]:
csv_agent = create_csv_agent(ChatGroq(temperature=0,
   model="llama3-70b-8192"), 
  'train_expanded.csv',
   verbose=True,
   agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
   allow_dangerous_code=True
   )

csv_agent.run('Can I order a product for delivery to a different country?')



> Entering new AgentExecutor chain...
Thought: I need to check if there's any information in the dataframe that suggests whether they deliver to different countries or not.

Action: python_repl_ast
Action Input: df.columnsIndex(['question', 'answer'], dtype='object')The observation shows that the dataframe has two columns: 'question' and 'answer'. There is no column that explicitly mentions delivery countries or international shipping.

Action: python_repl_ast
Action Input: df['answer'].str.contains('country|international|shipping')0      False
1      False
2      False
3      False
4      False
       ...  
195    False
196    False
197    False
198    False
199    False
Name: answer, Length: 200, dtype: boolThe observation shows that none of the answers in the dataframe contain the words 'country', 'international', or 'shipping', which suggests that there is no explicit information about delivering products to different countries.

Action: python_repl_ast
Action Input: df          

'The dataframe does not provide any information about delivering products to different countries, so it is unclear whether they deliver to different countries or not.'